In [107]:
import minio
import ldap3
from dotenv import dotenv_values
import logging

In [108]:
ldapSetupLogger = logging.getLogger('ldapSetup')

In [109]:
configValues = dotenv_values(dotenv_path="./setup.env")

# create an admin connection to ldap3
ldapServerURI = f"ldap://{configValues['FAIRSCAPE_LDAP_HOST']}:{configValues['FAIRSCAPE_LDAP_PORT']}" 
#ldapServer = ldap3.Server(ldapServerURI)

# for tests
ldapServerURI = "ldap://localhost:1389"


ldapBaseDN = configValues['FAIRSCAPE_LDAP_BASE_DN']
ldapUsersDN = configValues['FAIRSCAPE_LDAP_USERS_DN']
ldapGroupsDN = configValues['FAIRSCAPE_LDAP_GROUPS_DN']

configAdminDN = configValues['FAIRSCAPE_LDAP_CONFIG_ADMIN_DN']
configAdminPassword = configValues['FAIRSCAPE_LDAP_CONFIG_ADMIN_PASSWORD']

adminDN = configValues['FAIRSCAPE_LDAP_ADMIN_DN']
adminPassword = configValues['FAIRSCAPE_LDAP_ADMIN_PASSWORD']


In [110]:
configValues['FAIRSCAPE_LDAP_CONFIG_ADMIN_DN']

'cn=configadmin,cn=config'


`docker run --restart always --name openldap --env-file ./deploy/ldap.env -p 1389:1389 bitnami/openldap:latest`

LDAP_SKIP_DEFAULT_TREE="no"

In [111]:
# exceptions

# connect as the config admin
class LDAPConnectionError(Exception):
	def __init__(self,exception: Exception | None=None, message: str="failed to connect to ldap"):

		self.message = message
		self.exception = exception
		super().__init__(self.message)


In [112]:
# helper utilities

def connectLDAP(userDN, userPassword):
	try:
		server = ldap3.Server(ldapServerURI, get_info=ldap3.ALL)
		connection = ldap3.Connection(server,          
			user=userDN, 
			password=userPassword,
			lazy=False
			) 
		bind_response = connection.bind()
		if bind_response:
			ldapSetupLogger.info(f"msg: 'connected to LDAP server'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		return connection
	
	except ldap3.core.exceptions.LDAPBindError as bindError:
		ldapSetupLogger.error(f"msg: 'ldap bind error'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		raise LDAPConnectionError(
			exception=bindError, 
			message="ldap bind error"
			) 

	except ldap3.core.exceptions.LDAPSocketOpenError as connError:
		ldapSetupLogger.error(f"msg: 'ldap failed to open connection'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		raise LDAPConnectionError(
			exception=connError, 
			message=f"ldap connection error: failed to open connection at {ldapServerURI}"
			) 

	except ldap3.core.exceptions.LDAPException as ldapException:
		ldapSetupLogger.error(f"msg: 'ldap exception'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		raise LDAPConnectionError(
			exception=ldapException, 
			message=f"ldap error: {str(ldapException)}"
			)

In [113]:
def setupOverlay():
    # connect as the config admin
    configAdminConnection =connectLDAP(
        userDN=configAdminDN,
        userPassword=configAdminPassword
        )

    # add the module list to the config database 
    addModuleList = configAdminConnection.add(
        dn="cn=module,cn=config",
        attributes={
            "objectClass": "olcModuleList",
            "olcModuleLoad": [ "memberof.so", "refint.so"],
            "olcModulePath": "/opt/bitnami/openldap/lib/openldap"
        }
    )

    if not addModuleList:
        ldapSetupLogger.error(msg="msg: failed to add module list to config database")
        raise Exception("Setup Failed")

    configAdminConnection.rebind()

    try:
        ldap3.ObjectDef(['olcMemberOf'], configAdminConnection)

    except ldap3.core.exceptions.LDAPSchemaError as schemaError:
        ldapSetupLogger.error("failed to find olcMemberOf schema")
        raise Exception("Setup Failed")


    memberOfOverlayDN="olcOverlay={0}memberof,olcDatabase={2}mdb,cn=config"
    memberOfOverlayAttributes={
        "objectClass": [ "olcOverlayConfig", "olcMemberOf"],
        "olcOverlay": "memberof",
        "olcMemberOfRefInt": "TRUE",
    }

    addOverlay = configAdminConnection.add(
        dn=memberOfOverlayDN,
        attributes=memberOfOverlayAttributes,
    )


    configAdminConnection.rebind()

    if not addOverlay:
        overlayAppliedResult = configAdminConnection.search(
            search_base="olcDatabase={2}mdb,cn=config",
            search_scope=ldap3.SUBTREE,
            search_filter="(objectClass=olcOverlayConfig)"
        )

        if not overlayAppliedResult:
            ldapSetupLogger.error("failed to find applied memberOf overlay")
            raise Exception("ldap setup failure")

In [114]:
setupOverlay()

In [100]:
def setupFairscape():
	pass




True

In [ ]:
# if already exists raises
# ldap3.core.exceptions.LDAPEntryAlreadyExistsResult

In [ ]:
def getConfig():
	configValues = dotenv_values(dotenv_path="./setup.env")